# Competencia 2 NLP : Reconocimiento de entidades (NER) en Español

**Integrantes:** Alexander Cuevas - Jorge Gutiérrez - Benjamín Mellado - Felipe Urrutia

**Usuario del equipo en CodaLab:** *teamGalactico 🌌*

**Fecha límite de entrega 📆:** 29 de Junio a las 23:59 hrs.

**Tiempo estimado de dedicación:** En curso... ⏲️

**Link competencia:** [CC6205 Assignment 2 - Named Entity Recognition (NER) in Spanish](https://codalab.lisn.upsaclay.fr/competitions/5098?secret_key=09955d45-6210-4a35-a171-8050aa050855#learn_the_details)


## Introducción 🏷️

**REQUIREMENTS: Presentar brevemente el contexto, problema a resolver, incluyendo la formalización de la task (cómo son los inputs y outputs del problema) y los desafíos que ven al analizar el corpus entregado. (0.5 puntos)**

El análisis de documentos clínicos es una labor que permite identificar enfermedades, procedimientos, medicamentos, y partes del cuerpo, entre otras entidades relevantes. Además, se puede usar para determinar las relaciones entre cada una de estas. Con estos datos es posible elaborar estadísticas para una población y sus individuos. En particular, se pueden determinar factores de riesgo, priorizar pacientes de acuerdo a sus necesidades, establecer casos solucionables sin intervención presencial, conocer antecedentes familiares, saber cuantas personas sufren más de una enfermedad, cuantas consumen más de un medicamento, etc.

En Chile, el sistema público de salud cubre al 75% de la población. Bajo este sistema, las visitas a especialistas, que requieren la remisión de un médico general, son manejadas a través de una lista de espera dividida para casos GES y NO GES. En 2017, hubo 1.661.826 personas esperando una cita con un especialista, con un promedio de espera por sobre los 400 días. Por este motivo, se hace necesario desarrollar sistemas automatizados que analicen la lista NO GES, para mejorar el manejo de pacientes. En particular, existe una falta de trabajos realizados en el área de NER en Español y aún más en el contexto clínico.

Bajo tal contexto, en este trabajo utilizaremos el Corpus de Listas de Espera propuesto por [1]. Este corpus está constituido por derivaciones de diversas consultas de la lista de espera NO GES de los hospitales públicos chilenos. Por su naturaleza, cada una de estas derivaciones es un dato no-estructurado escrito en lenguaje natural. Entre los resultados principales de [1] está otorgar un Corpus anotado usando texto clínico en español.

La tarea principal propuesta por [1] es un reconocimiento de entidades nombradas. También conocida como NER, por sus siglas en ingles. Dicha tarea es una de las más populares en NLP. La cual consiste en identificar automáticamente piezas esenciales de information (entidades) en el texto. Tradicionalmente, la categoría de entidades a identificar eran nombres de personas, lugares y organizaciones. Actualmente, los tipos de categorías se han expandido a diversos dominios del conocimiento. En este trabajo, las entidades que estudiaremos viven en el contexto de texto clínico.

[1] utiliza el esquema Unified Medical Language System (UMLS) para las categorías de entidades. Originalmente, [1] genera un Corpus con siete tipos de entidades:

    (1) Finding, (2) Procedure, (3) Family Member, (4) Disease, (5) Body Part, (6) Medication y (7) Abbreviation. 
    
Sin embargo, en este trabajo solo estudiaremos cinco de ellas. Todas excepto (1) y (7). Las anotaciones siguen el esquema IOB2, estandar en NER. Esquema definido como sigue: 

>*Esquema IOB2*. Consiste en identificar cada token con una etiqueta que indica si pertenece o no a una entidad. Si un token **no** es parte de una entidad, se indican con $\texttt{O}$ (por la inicial en Ingles de *Fuera*). Sino, este necesariamente es un token de una entidad. Para indicar si el token es el inicio de una entidad, se utiliza el prefijo $\texttt{B-}$ (por la inicial en Ingles de *Comienzo*). En cambio, si el token es la continuacion de una entidad, se utiliza el prefijo $\texttt{I-}$ (por la inicial en Ingles de *Dentro*). Un ejemplo de este esquema, es el siguiente: 

\begin{align*}
&\texttt{PRESENTA}& &\texttt{FRACTURA}& &\texttt{CORONARIA}& &\texttt{COMPLICADA}& &\texttt{EN}& &\quad\text{(Texto)}\\
&\texttt{O}& &\texttt{B-Disease}& &\texttt{I-Disease}& &\texttt{I-Disease}& &\texttt{O}& &\quad\text{(IOB2)}
\end{align*}

Tal como se puede observar del ejemplo anterior, si se sigue el esquema de anotamiento IOB2 para la tarea de NER, el problema de reconocimiento de entidades nombradas se transforma en un problema de Sequence-Labeling. Tipo de problema que hemos discutido durante las últimas unidades del curso. Entre los modelos estudiados estan HMM, CRF, MEMMS, CNN y RNN. En este trabajo, nos focalizaremos en solo modelos neuronales dentro de la familia de las Redes Neuronales Recurrentes (RNN, por sus siglas en Ingles).

Ahora bien, el conjunto de datos vienen entregados en una forma estándar para la tarea de NER, denominado ConLL. Este consiste en un archivo de texto con dos columnas y saltos de línea que separan cada secuencia. La primera columna contiene los tokens de la secuencia. Mientras que la segunda columna, contiene las etiquetas IOB2 asociadas a cada token de la secuencia. Por otro lado, los datos vienen previamente separados en conjuntos de Entrenamiento, Desarrollo y Prueba. Estos conjuntos tienen tamaños X, Y y Z, resp. Se espera que el output siga el mismo formato, y resulte de modificar el conjunto de Prueba que contiene tokens con todas las entidades inicializadas en $\texttt{O}$.

Además, la proporción de etiquetas IOB2 son mayoritariamente $\texttt{O}$. Con una proporción total de A, B y C, en cada conjunto resp. El segundo tipo de etiqueta IOB2 más frecuente es D, con proporciones E, F y G. Mientras que la etiqueta IOB2 menos frecuente es H, con proporciones I, J, K. Esta diferencia considerable de proporciones de las etiquetas, se denomina desbalance de datos.

Entre las dificultades encontradas para esta tarea están: (1) Desbalance de etiquetas IOB2, (2) [alguna otra encontrada del análisis exploratorio de datos] y (3) NER en contexto de texto clínico. Consideramos que dichas dificultades son desafiantes de resolver. Es por esto que es de nuestro interés lograr resolver satisfactoriamente esta tarea. En las secciones que siguen se encuentra detalladamente nuestra solución.

**Referencias**

[1] Báez, P., Villena, F., Rojas, M., Durán, M., & Dunstan, J. (2020, November). The Chilean Waiting List Corpus: a new resource for clinical named entity recognition in Spanish. In Proceedings of the 3rd clinical natural language processing workshop (pp. 291-300). https://aclanthology.org/2020.clinicalnlp-1.32/

## Modelos 🤖

**REQUIREMENTS: Describir brevemente los modelos, métodos e hiperparámetros utilizados. (1.0 puntos)**
### Baseline

El primer modelo probado fue el entregado como baseline para la competencia. Este consiste en una red neuronal no bidireccional con capa de embedding, tres capas LSTM, y una capa de salida. Utiliza dropout en el entrenamiento. Como hiperparámetros, se fija la dimensión de los embeddings a 200, y la de las tres capas ocultas a 128. Además, la cantidad de épocas de entrenamiento se fija en 10. Finalmente, se define la función de pérdida como la entropía cruzada entre el input y los datos objetivo.

### Sin stopwords

### Con embeddings clínicos del lenguaje español

### Con embeddings generales del lenguaje español
En adición a los embeddings clínicos, se realizaron pruebas con embeddings generales del lenguaje español, en particular, cinco conjuntos de embeddings disponibles en la página [Spanish Word Embeddings](https://github.com/dccuchile/spanish-word-embeddings) (consultado el 11 de junio de 2022). Estos provienen de tres corpus de origen: Spanish Unannotated Corpora (SUC), Spanish Billion Word Corpus (SWBC), y Spanish Wikipedia. Y fueron procesados usando tres algoritmos: FastText, Glove, y Word2Vec.

Para cada conjunto anterior se creó un modelo usando el baseline como base. Para todos estos se cambió el hiperparámetro de dimensión de embeddings a 300, y se mantuvo el resto igual a lo reportado anteriormente. Además, se modificó la capa de embeddings según correspondiera para utilizar los embeddings de la página en formato .vec.

## Baseline mejorado
Luego de probar los embeddings generales, se procedió a ajustar los parámetros del modelo baseline. En particular, se ajustó la dimensión de entrada del embedding usado, para que corresponda al que dio mejor resultado: Glove aplicado a SWBC (855380).

Adicionalmente, se usaron capas LSTM de 256 dimensiones. Dropout igual a 0.5, y con bidireccionalidad. Para la predicción, se entrenó el modelo 3 veces.

### Con datos balanceados

## Métricas de evaluación 📊

**REQUIREMENTS: Describir las métricas utilizadas en la evaluación indicando qué miden y cuál es su interpretación en este problema en particular. (0.5 puntos)**


En un problema de Sequence-labeling, la variable predicha por un modelo predictivo es un vector de etiquetas, donde cada etiqueta está asociada a un token en la secuencia del input. Una metrica usual para medir el desempeño de un modelo de este tipo determina TP, TN, FP y FN con cada etiqueta independientemente. Sin embargo, en la tarea de NER, el objetivo es reconocer entidades nombradas en el texto. Siendo las entidades, las predicciones importantes del modelo y no las etiquetas individuales en formato IOB2. Esto porque en el contexto clínico, varias condiciones y medicamentos tienen asociados una serie de adjetivos, biológicos y químicos, que afectan su significado e intensidad. Por su parte, los detalles de un paciente, y la especificación de partes del cuerpo son similarmente relevantes. Por esto, una metrica usual no cuantifica el desempeño del modelo para el problema original. Para ser más preciso, existen dos motivos, (1) las etiquetas importantes son aquellas asociadas a las entidades y (2) el modelo debe de reconocer exactamente tanto la ubicación de la entidad como el tipo de entidad. Una metrica que si logra estas dos cosas, se denomina **metrica estricta**.

<!-- \begin{align*}
&\texttt{PRESENTA}& &\texttt{FRACTURA}& &\texttt{CORONARIA}& &\texttt{COMPLICADA}& &\texttt{EN}& &\quad\text{(Texto)}\\
&\texttt{O}& &\texttt{B-Disease}& &\texttt{I-Disease}& &\texttt{I-Disease}& &\texttt{O}& &\quad\text{(IOB2)}\\
&0& &1& &2& &3& &4& &\quad\text{(Indice)}
\end{align*} -->

En lo que sigue detallaremos las métricas estrictas utilizadas. Para formalizar su definición, consideremos $D = (S, T)$ un conjunto de $n$ datos. Donde para cada secuencia $s \in S$, $t^s \in T$ corresponden a las etiquetas de $s$. Sin perdida de generalidad, supondremos que toda secuencia es de largo $m$. En la práctica, $m$ corresponde al largo máximo de una secuencia dentro del conjunto de datos. Si una secuencia (y sus etiquetas) es de largo menor que $m$, se rellena con $\texttt{<pad>}$. Consideremos $P$ a las etiquetas predichas, i.e., $p^s \in P$ es la predicción de la secuencia $s \in S$, comparable con las etiquetas reales $t^s \in T$. Naturalmente, $s_i$ es el i-esimo token de la secuencia $s$. Análogamente, $t^s_i$ para su etiqueta real y $p^s_i$ para la predicha. Se dirá que $(l, i, j, c)$ es una entidad válida de tipo $c$ en las etiquetas $l$, si $i\leq j$, $l_i = \texttt{B-}c$, para cada $k \in \{i+1, ..., j\}$, $l_k = \texttt{I-}c$ y, si $j<m$, entonces $l_{j+1} \neq \texttt{I-}c$. Denotaremos por $E(l, c)$ al conjunto de las entidades válidas de tipo $c$ en las etiquetas $l$. Adicionalmente, denotaremos por $E(L)$ al conjunto de todas las entidades válidas según $L$, i.e.,  $$E(L) = \bigcup \{E(l,  c): l \in L \text{ y $c$ es un tipo de entidad}\}$$
  
* *Precision*. Para un tipo de entidad fija (p.ej. Disease), la precision mide la proporción de veces que una entidad predicha de tal tipo coincide con la entidad real sobre el total de veces que el modelo predice una entidad con el tipo deseado. En efecto, para una entidad de tipo $c$,

$$ \texttt{Precision}(c) = \frac{|\{ (l, i, j, d) \in E(P): d = c, \exists s \in S, l = p^s, (p^s, i, j, c) \in E(t^s, c)\}|}{|(l, i, j, d) \in E(P): d=c |}$$

* *Recall*. En el contexto clínico, el recall también se conoce como sensibilidad, y es importante dado que frente a diagnósticos y medicamentos sensibles, es preferible descartar un caso FN a no haberlo considerado en primer lugar. Para un tipo de entidad fija, el recall mide la proporción de veces que una entidad predicha de tal tipo coincide con la entidad real sobre el total de entidades del tipo deseado. En efecto, para una entidad de tipo $c$,

$$ \texttt{Recall}(c) = \frac{|\{ (l, i, j, d) \in E(P): d = c, \exists s\in S, l=p^s, (p^s, i, j, c) \in E(t^s, c)\}|}{|(l, i, j, d) \in E(T): d=c |}$$

* *Micro F1 score*. **Recuerde hacer la distinción entre lo que sería una métrica de micro f1-score vs macro f1-score.** Para un tipo de entidad fija, la Precision y el Recall, entregan información distinta sobre las predicciones para dicho tipo de entidad. El primero considera falsos-positivos, mientras que el segundo considera falsos-negativos. Una forma de condensar esta información en una única métrica es con el promedio armónico entre Precision y Recall, denominado F1-score. Dicha metrica es condicional al tipo de entidad elegida. Para obtener un F1-score global (que considere todos los tipos de entidades), existen dos aproximaciones: (1) Micro F1-score y (2) Macro F1-score. En palabras simples, (1) es el Accuracy del modelo y (2) Un promedio de F1-score por tipo de entidad. Descartamos Macro F1-score dado que nuestros datos son desbalancedados. Por lo tanto, cada F1-score por clase no son estadísticamente comparables. Nos quedaremos con Micro F1-score, ya que traduce globalmente el desempeño del modelo. Para computar esta metrica, se utiliza la técnica estricta definida anteriormente. En efecto,

$$ \texttt{Micro-F1-score} = \frac{|\{ (l, i, j, d) \in E(P): \exists s\in S, l=p^s, (p^s, i, j, d) \in E(t^s, d)\}|}{|S|}$$



## Diseño experimental ✏️

**REQUIREMENTS: Esta es una de las secciones más importantes del reporte. Deben describir minuciosamente los experimentos que realizarán en la siguiente sección. Describir las variables de control que manejarán, algunos ejemplos pueden ser: Los hiperparámetros de los modelos, tipo de embeddings utilizados, tipos de arquitecturas. Ser claros con el conjunto de hiperparámetros que probarán, la decisión en las funciones de optimización, función de pérdida, regulación, etc. Básicamente explicar qué es lo que veremos en la siguiente sección. (1 punto)**

### Con embeddings generales del lenguaje español
En adición a los embeddings clínicos, se realizaron pruebas con embeddings generales del lenguaje español, en particular, los cinco conjuntos de embeddings disponibles en la página: [Spanish Word Embeddings](https://github.com/dccuchile/spanish-word-embeddings)
### Hiperparámetros
Estos son comunes para los cinco conjuntos.

    Arquitectura: NER RNN
    Dimensión de entrada: Largo del vocabulario de entrenamiento (17591)
    Dimensión de salida: Número de clases (12)
    Tipo de capas intermedias: LSTM
    Número de capas intermedias: 3
    Dimensión de capas intermedias: 256
    Tamaño de los batches: 16
    Función de pérdida: CrossEntropyLoss
    Optimizador: Algoritmo ADAM
    Bidireccional: Sí
    Dropout: 0.5
    Número de epochs: 10

#### Fast SWBC
    Capa de embedding: Pre-entrenada (855380x300)

#### Glove SWBC
    Capa de embedding: Pre-entrenada (855380x300)

#### Fast SUC
    Capa de embedding: Pre-entrenada (1313423x300)

#### W2V SWBC
    Capa de embedding: Pre-entrenada (1000653x300)

#### Wikipedia (español)
    Capa de embedding: Pre-entrenada (985667x300)

## Baseline mejorado
Luego de probar los embeddings generales y ver que no existía una mejora notable, se procedió a probar con los parámetros ajustados del modelo baseline, pero sin los embeddings.
### Hiperparámetros
    Arquitectura: NER RNN
    Dimensión de entrada: Largo del vocabulario de entrenamiento (17591)
    Dimensión de salida: Número de clases (12)
    Tipo de capas intermedias: LSTM
    Número de capas intermedias: 3
    Dimensión de capas intermedias: 256
    Tamaño de los batches: 16
    Capa de embedding: Sin pre-entrenar (855380x300)
    Función de pérdida: CrossEntropyLoss
    Optimizador: Algoritmo ADAM
    Bidireccional: Sí
    Dropout: 0.5
    Número de epochs: 10

## Experimentos 🧪

**REQUIREMENTS: Reportar todos sus experimentos y código en esta sección. Comparar los resultados obtenidos utilizando diferentes modelos. ¡Es vital haber realizado varios experimentos para sacar una buena nota! (2.0 puntos)**

### Configuración común

In [ ]:
%pip install -U torchtext==0.10.0
%pip install seqeval

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from seqeval.metrics import f1_score, precision_score, recall_score
from torchtext import data, datasets, legacy

# Garantizar reproducibilidad de los experimentos
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT = legacy.data.Field(lower=False) 
NER_TAGS = legacy.data.Field(unk_token=None)
fields = (("text", TEXT), ("nertags", NER_TAGS))


train_data, valid_data, test_data = legacy.datasets.SequenceTaggingDataset.splits(
    path="./",
    train="train.txt",
    validation="dev.txt",
    test="test.txt",
    fields=fields,
    encoding="utf-8",
    separator=" "
)


print(f"Numero de ejemplos de entrenamiento: {len(train_data)}")
print(f"Número de ejemplos de validación: {len(valid_data)}")
print(f"Número de ejemplos de test (competencia): {len(test_data)}")


TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']


def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages


print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

BATCH_SIZE = 16  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test. Si van a hacer algún sort no puede ser sobre
# el conjunto de testing ya que al hacer sus predicciones sobre el conjunto de test sin etiquetas
# debe conservar el orden original para ser comparado con los golden_labels. 

train_iterator, valid_iterator, test_iterator = legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)


def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)

    # filtramos <pad> para calcular los scores.
    mask = [(y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu').numpy()
    y_true = y_true.to('cpu').numpy()
    y_pred = [[NER_TAGS.vocab.itos[v] for v in y_pred]]
    y_true = [[NER_TAGS.vocab.itos[v] for v in y_true]]
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, mode='strict')
    precision = precision_score(y_true, y_pred, mode='strict')
    recall = recall_score(y_true, y_pred, mode='strict')

    return precision, recall, f1

### Embeddings generales del español

In [ ]:
import torch.nn as nn
import numpy as np
import gensim

# Cargar data de los embeddings manualmente
embedding_glove_swbc_dict = dict()
embedding_fast_swbc_dict = dict()
embedding_fast_suc_dict = dict()
embedding_w2v_dict = dict()
embedding_wiki_dict = dict()

for par in [
            [embedding_glove_swbc_dict, 'glove-sbwc.i25.vec'],
            [embedding_fast_swbc_dict, 'fasttext-sbwc.3.6.e20.vec'],
            [embedding_fast_suc_dict, 'embedding-l-model.vec'],
            [embedding_w2v_dict, 'SBW-vectors-300-min5.vec']
            [embedding_wiki_dict, 'wiki.es.vec']
            ]:
  with open(par[1], mode='rb') as f:
    first = True
    for line in f:
      if first:
        first = False
        continue
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      par[0][word] = coefs
  print('Loaded %s word vectors.' % len(par[0]))

In [ ]:
# Cargar embeddings con gensim
temp = gensim.models.KeyedVectors.load_word2vec_format('glove-sbwc.i25.vec')
embedding_glove_swbc = nn.Embedding.from_pretrained(torch.FloatTensor(temp.vectors))

temp = gensim.models.KeyedVectors.load_word2vec_format('fasttext-sbwc.3.6.e20.vec')
embedding_fast_swbc = nn.Embedding.from_pretrained(torch.FloatTensor(temp.vectors))

temp = gensim.models.KeyedVectors.load_word2vec_format('embedding-l-model.vec')
embedding_fast_suc = nn.Embedding.from_pretrained(torch.FloatTensor(temp.vectors))

temp = gensim.models.KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.vec')
embedding_w2v = nn.Embedding.from_pretrained(torch.FloatTensor(temp.vectors))

temp = gensim.models.KeyedVectors.load_word2vec_format('wiki.es.vec')
embedding_wiki = nn.Embedding.from_pretrained(torch.FloatTensor(temp.vectors))

Fast SUC

In [ ]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout):

        super().__init__()

        # Capa de embedding
        self.embedding = embedding_fast_suc

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(self.dropout(outputs))
        return predictions


# Tamaño del vocabulario
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = True

baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...
baseline_n_epochs = 10


# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)


model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs


def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


model.apply(init_weights)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')


optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]



        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [ ]:
best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

Epoch: 01 | Epoch Time: 4m 1s
	Train Loss: 0.935 | Train f1: 0.30 | Train precision: 0.53 | Train recall: 0.21
	 Val. Loss: 0.759 |  Val. f1: 0.45 |  Val. precision: 0.70 | Val. recall: 0.34
Epoch: 02 | Epoch Time: 4m 7s
	Train Loss: 0.746 | Train f1: 0.46 | Train precision: 0.65 | Train recall: 0.36
	 Val. Loss: 0.639 |  Val. f1: 0.51 |  Val. precision: 0.72 | Val. recall: 0.40
Epoch: 03 | Epoch Time: 11m 35s
	Train Loss: 0.662 | Train f1: 0.52 | Train precision: 0.69 | Train recall: 0.42
	 Val. Loss: 0.586 |  Val. f1: 0.55 |  Val. precision: 0.74 | Val. recall: 0.45
Epoch: 04 | Epoch Time: 3m 56s
	Train Loss: 0.604 | Train f1: 0.56 | Train precision: 0.72 | Train recall: 0.47
	 Val. Loss: 0.564 |  Val. f1: 0.59 |  Val. precision: 0.75 | Val. recall: 0.50
Epoch: 05 | Epoch Time: 3m 57s
	Train Loss: 0.566 | Train f1: 0.59 | Train precision: 0.73 | Train recall: 0.50
	 Val. Loss: 0.546 |  Val. f1: 0.59 |  Val. precision: 0.76 | Val. recall: 0.49
Epoch: 06 | Epoch Time: 3m 58s
	Train Los

In [ ]:
model.load_state_dict(torch.load('{}.pt'.format(model_name)))
torch.cuda.empty_cache()

valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.483 |  Val. f1: 0.64 | Val. precision: 0.75 | Val. recall: 0.56


Fast SWBC

In [ ]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout):

        super().__init__()

        # Capa de embedding
        self.embedding = embedding_fast_swbc

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(self.dropout(outputs))
        return predictions


# Tamaño del vocabulario
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = True

baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...
baseline_n_epochs = 10


# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)


model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs


def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


model.apply(init_weights)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')


optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]



        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [ ]:
best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

c:\Users\autofact\Documents\NLP-competition-2\venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\autofact\Documents\NLP-competition-2\venv\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 3m 59s
	Train Loss: 0.944 | Train f1: 0.28 | Train precision: 0.50 | Train recall: 0.20
	 Val. Loss: 0.767 |  Val. f1: 0.43 |  Val. precision: 0.69 | Val. recall: 0.32
Epoch: 02 | Epoch Time: 3m 56s
	Train Loss: 0.751 | Train f1: 0.46 | Train precision: 0.66 | Train recall: 0.36
	 Val. Loss: 0.639 |  Val. f1: 0.50 |  Val. precision: 0.74 | Val. recall: 0.39
Epoch: 03 | Epoch Time: 3m 57s
	Train Loss: 0.674 | Train f1: 0.51 | Train precision: 0.69 | Train recall: 0.41
	 Val. Loss: 0.605 |  Val. f1: 0.55 |  Val. precision: 0.70 | Val. recall: 0.45
Epoch: 04 | Epoch Time: 4m 12s
	Train Loss: 0.612 | Train f1: 0.56 | Train precision: 0.72 | Train recall: 0.46
	 Val. Loss: 0.563 |  Val. f1: 0.58 |  Val. precision: 0.74 | Val. recall: 0.48
Epoch: 05 | Epoch Time: 4m 13s
	Train Loss: 0.566 | Train f1: 0.59 | Train precision: 0.73 | Train recall: 0.50
	 Val. Loss: 0.532 |  Val. f1: 0.61 |  Val. precision: 0.74 | Val. recall: 0.54
Epoch: 06 | Epoch Time: 4m 5s
	Train Los

In [ ]:
model.load_state_dict(torch.load('{}.pt'.format(model_name)))
torch.cuda.empty_cache()

valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.460 |  Val. f1: 0.66 | Val. precision: 0.73 | Val. recall: 0.61


Glove SWBC

In [ ]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout):

        super().__init__()

        # Capa de embedding
        self.embedding = embedding_glove_swbc

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(self.dropout(outputs))
        return predictions


# Tamaño del vocabulario
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = True

baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...
baseline_n_epochs = 10


# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)


model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs


def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


model.apply(init_weights)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')


optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]



        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [ ]:
best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

c:\Users\autofact\Documents\NLP-competition-2\venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\autofact\Documents\NLP-competition-2\venv\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 3m 59s
	Train Loss: 0.944 | Train f1: 0.28 | Train precision: 0.50 | Train recall: 0.20
	 Val. Loss: 0.767 |  Val. f1: 0.43 |  Val. precision: 0.69 | Val. recall: 0.32
Epoch: 02 | Epoch Time: 3m 56s
	Train Loss: 0.751 | Train f1: 0.46 | Train precision: 0.66 | Train recall: 0.36
	 Val. Loss: 0.639 |  Val. f1: 0.50 |  Val. precision: 0.74 | Val. recall: 0.39
Epoch: 03 | Epoch Time: 3m 57s
	Train Loss: 0.674 | Train f1: 0.51 | Train precision: 0.69 | Train recall: 0.41
	 Val. Loss: 0.605 |  Val. f1: 0.55 |  Val. precision: 0.70 | Val. recall: 0.45
Epoch: 04 | Epoch Time: 4m 12s
	Train Loss: 0.612 | Train f1: 0.56 | Train precision: 0.72 | Train recall: 0.46
	 Val. Loss: 0.563 |  Val. f1: 0.58 |  Val. precision: 0.74 | Val. recall: 0.48
Epoch: 05 | Epoch Time: 4m 13s
	Train Loss: 0.566 | Train f1: 0.59 | Train precision: 0.73 | Train recall: 0.50
	 Val. Loss: 0.532 |  Val. f1: 0.61 |  Val. precision: 0.74 | Val. recall: 0.54
Epoch: 06 | Epoch Time: 4m 5s
	Train Los

In [ ]:
model.load_state_dict(torch.load('{}.pt'.format(model_name)))
torch.cuda.empty_cache()

valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.460 |  Val. f1: 0.66 | Val. precision: 0.73 | Val. recall: 0.61


Wiki es

In [ ]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout):

        super().__init__()

        # Capa de embedding
        self.embedding = embedding_wiki

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(self.dropout(outputs))
        return predictions


# Tamaño del vocabulario
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = True

baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...
baseline_n_epochs = 10


# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)


model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs


def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


model.apply(init_weights)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')


optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]



        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [ ]:
best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

c:\Users\autofact\Documents\NLP-competition-2\venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\autofact\Documents\NLP-competition-2\venv\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 3m 55s
	Train Loss: 0.937 | Train f1: 0.30 | Train precision: 0.53 | Train recall: 0.22
	 Val. Loss: 0.748 |  Val. f1: 0.44 |  Val. precision: 0.68 | Val. recall: 0.33
Epoch: 02 | Epoch Time: 4m 5s
	Train Loss: 0.750 | Train f1: 0.46 | Train precision: 0.66 | Train recall: 0.36
	 Val. Loss: 0.645 |  Val. f1: 0.49 |  Val. precision: 0.73 | Val. recall: 0.37
Epoch: 03 | Epoch Time: 3m 56s
	Train Loss: 0.666 | Train f1: 0.51 | Train precision: 0.68 | Train recall: 0.41
	 Val. Loss: 0.589 |  Val. f1: 0.56 |  Val. precision: 0.72 | Val. recall: 0.47
Epoch: 04 | Epoch Time: 4m 3s
	Train Loss: 0.612 | Train f1: 0.56 | Train precision: 0.72 | Train recall: 0.46
	 Val. Loss: 0.570 |  Val. f1: 0.56 |  Val. precision: 0.74 | Val. recall: 0.47
Epoch: 05 | Epoch Time: 6m 41s
	Train Loss: 0.568 | Train f1: 0.59 | Train precision: 0.74 | Train recall: 0.50
	 Val. Loss: 0.538 |  Val. f1: 0.59 |  Val. precision: 0.75 | Val. recall: 0.50
Epoch: 06 | Epoch Time: 4m 2s
	Train Loss:

In [ ]:
model.load_state_dict(torch.load('{}.pt'.format(model_name)))
torch.cuda.empty_cache()

valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.469 |  Val. f1: 0.66 | Val. precision: 0.74 | Val. recall: 0.60


### Baseline mejorado

In [ ]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(855380, embedding_dim)

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions


# Tamaño del vocabulario
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = True

baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...
baseline_n_epochs = 10


# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)


model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs


def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)


model.apply(init_weights)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')


optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]



        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [ ]:
best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 1m 5s
	Train Loss: 0.758 | Train f1: 0.45 | Train precision: 0.61 | Train recall: 0.38
	 Val. Loss: 0.465 |  Val. f1: 0.65 |  Val. precision: 0.77 | Val. recall: 0.57
Epoch: 02 | Epoch Time: 1m 6s
	Train Loss: 0.393 | Train f1: 0.72 | Train precision: 0.77 | Train recall: 0.68
	 Val. Loss: 0.397 |  Val. f1: 0.71 |  Val. precision: 0.80 | Val. recall: 0.65
Epoch: 03 | Epoch Time: 1m 6s
	Train Loss: 0.265 | Train f1: 0.81 | Train precision: 0.83 | Train recall: 0.79
	 Val. Loss: 0.365 |  Val. f1: 0.76 |  Val. precision: 0.78 | Val. recall: 0.75
Epoch: 04 | Epoch Time: 1m 7s
	Train Loss: 0.192 | Train f1: 0.86 | Train precision: 0.87 | Train recall: 0.86
	 Val. Loss: 0.343 |  Val. f1: 0.77 |  Val. precision: 0.79 | Val. recall: 0.75
Epoch: 05 | Epoch Time: 1m 6s
	Train Loss: 0.151 | Train f1: 0.89 | Train precision: 0.89 | Train recall: 0.89
	 Val. Loss: 0.357 |  Val. f1: 0.77 |  Val. precision: 0.79 | Val. recall: 0.76
Epoch: 06 | Epoch Time: 1m 6s
	Train Loss: 0.

In [ ]:
model.load_state_dict(torch.load('{}.pt'.format(model_name)))
torch.cuda.empty_cache()

valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(model, valid_iterator, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.331 |  Val. f1: 0.77 | Val. precision: 0.80 | Val. recall: 0.75


## Conclusiones 💡

**REQUIREMENTS: Discutir resultados, proponer trabajo futuro. (1 punto)**